## Practical Exercises - Part 1

In [4]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn

seaborn.set_context('talk')

## Reading the dataset

In [7]:
url = 'https://www.famaf.unc.edu.ar/~nocampo043/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [8]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,profile_career,profile_university,profile_specialization_courses,profile_open_source_contributions,profile_programming_as_hobby,work_role,tools_platform,tools_programming_languages,tools_frameworks,tools_data_bases,tools_qa_testing,tools_IDEs,tools_work_pc_os,tools_cellphone_os,work_on_call_duty,salary_on_call_duty_charge,work_on_call_duty_charge_type,work_contract_type,salary_monthly_GROSS,salary_monthly_NET,salary_in_usd,salary_satisfaction,salary_comparison_last_semester,salary_has_bonus,salary_inflation_adjustment_2020,salary_percentage_inflation_adjustment_2020,salary_month_last_inflation_adjustment,work_has_violence_situations,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies
0,Female,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,University,Ongoing,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Yes, they were paid by my employer",No,Yes,Consultant,None,None,None,None,None,None,GNU/Linux,Android,No,0.0,Gross,Full-Time,53000.0,43000.0,NaN,1,1,No,No,0.0,0,In my current job,NaN,501-1000,Services / Software Consulting / Digital,7,2
1,Male,29,Argentina,Corrientes,5.0,2.0,2.0,4,University,Ongoing,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Yes, they were paid by myself, Yes, they were ...",No,Yes,Developer,"Docker, Google Cloud Platform/App Engine","HTML, Javascript, Python","Django, jQuery","Informix, MySQL, PostgreSQL",None,"IPython / Jupyter, Sublime Text, Visual Studio...",GNU/Linux,Android,No,0.0,Gross,Full-Time,79000.0,63000.0,NaN,2,2,Less than a salary,One,10.0,1,Never,No,201-500,Other industries,8,9
2,Female,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secondary,Complete,NaN,NaN,"Yes, they were paid by myself, Yes, they were ...",No,Yes,Infosec,"Linux, VMWare",None,None,None,None,None,Windows,iOS,No,0.0,Percentage of my salary,Full-Time,160000.0,127000.0,NaN,3,3,One salary,No,0.0,0,In a previous job,No,2001-5000,Other industries,6,9


# Exercise 1 - Descriptive analysis

Answer the question: **What programming languages are ​​associated with the best salaries?**

For it:
1. Select the relevant columns to analyze.
2. Select the relevant rows to analyze. This includes removing extreme and erroneous values, but you can also focus the analysis on a sub-population. For example, they can be limited to people with a salary greater than 10,000 pesos or people who work only in "Data Science", but you must justify your choice and reformulate the initial question if necessary.
  * Get a list of the most popular programming languages. Then, decide how many and which ones to select to include in the analysis.
  * For each of the columns from the previous point, choose the ranges or values ​​selected to include in the analysis.
3. Select metrics that help answer the question and methods to analyze them. Choose ONE of the following options:
  * Compare salary distributions for each language using visualizations. As the visualization is the final product, it must be clear and show relevant information.
  * Compare measures of descriptive statistics on salary distribution for each language. Be creative. Descriptive statistics allow us to say things like: "Kotlin programmers mostly earn the top 10% of salaries!" (where *majority* is a slippery middle ground that just means more than 50%). To compare multiple languages, we recommend using visualizations as well.
  * Compare odds. For example: "If you know Python or Java, you have a 30% more chance of winning over 100K".

If you consider it necessary, carry out several iterations. That is, if they find that the distributions of the programming languages ​​they initially selected are not very different, they can redo the analysis using only the programming languages ​​that are different.

In [14]:
# complete here if you want to include more columns
relevant_columns = ['tools_programming_languages', 'salary_monthly_NET']

### Frequency count of programming languages

The column that contains information about the programming languages used is `tools_programming_languages`. Its values are strings with the selected languages separated by commas.

In [15]:
df.tools_programming_languages[:3]

0                        None
1    HTML, Javascript, Python
2                        None
Name: tools_programming_languages, dtype: object

The following code cells separate these programming languages and count how often they appear.

It is not necessary to understand this code in depth, although it is a good exercise.

In [16]:
# Convert the comma-separated string of languages to a list of string.
# Remove 'None of the previous one' option, spaces and training commas.
def split_languages(languages_str):
  if not isinstance(languages_str, str):
    return []
  # Remove 'other' option
  languages_str = languages_str.lower()\
    .replace('None of the previous one', '')
  # Split string into list of items
  # Remove spaces and commas for each item
  return [lang.strip().replace(',', '')
          for lang in languages_str.split()]

# Create a new column with the list of languages
df.loc[:, 'cured_programming_languages'] = df.tools_programming_languages\
    .apply(split_languages)
if 'cured_programming_languages' not in relevant_columns:
    relevant_columns.append('cured_programming_languages') 

# Duplicate each row of df for each programming language
# mentioned in the response.
# We only include in df_lang the columns we are going to analyze later, so we
# don't duplicate innecesary information.
df_lang = df.cured_programming_languages\
    .apply(pd.Series).stack()\
    .reset_index(level=-1, drop=True).to_frame()\
    .join(df[relevant_columns])\
    .rename(columns={0: 'programming_language'})
# Horrible programming style! But a lot of data science code can be written with
# as concatenations of functions (pipelines), and there's no elegant way of
# doing that on Python.
df_lang[:5]

,programming_language,tools_programming_languages,salary_monthly_NET,cured_programming_languages
0,none,None,43000.0,[none]
1,html,"HTML, Javascript, Python",63000.0,"[html, javascript, python]"
1,javascript,"HTML, Javascript, Python",63000.0,"[html, javascript, python]"
1,python,"HTML, Javascript, Python",63000.0,"[html, javascript, python]"
2,none,None,127000.0,[none]


In the `programming_language` column, you will find each language separately. Note that if a response contained 3 languages, such as `"HTML, Javascript, Python"`, the row has been replicated 3 times. Therefore, there are three rows with index 1.

In [17]:
language_count = df_lang.programming_language.value_counts()\
    .reset_index()\
    .rename(columns={'index': 'language', 'programming_language': 'frequency'})
language_count[:10]

,language,frequency
0,javascript,2513
1,sql,2154
2,html,1861
3,python,1475
4,java,1453
5,css,1247
6,bash/shell,993
7,none,987
8,php,891
9,.net,823


## Relevant language filtering

The following code allows you to select only the rows where the value of the `programming_language` column is found in the `interesting_languages` list.

In [20]:
# Filter out languages that we want to exclude
# Complete here with your selected list.
interesting_languages = ["python"]
filtered_df_lang = df_lang[df_lang.programming_language.isin(interesting_languages)]
filtered_df_lang

,programming_language,tools_programming_languages,salary_monthly_NET,cured_programming_languages
1,python,"HTML, Javascript, Python",63000.0,"[html, javascript, python]"
3,python,"Bash/Shell, C, CSS, HTML, Java, Javascript, PH...",102000.0,"[bash/shell, c, css, html, java, javascript, p..."
9,python,"Assembler, Bash/Shell, C, C#, Python, R",39000.0,"[assembler, bash/shell, c, c#, python, r]"
10,python,"HTML, Javascript, Python",36800.0,"[html, javascript, python]"
11,python,"Bash/Shell, Python",94621.0,"[bash/shell, python]"
...,...,...,...,...
6085,python,"C#, CSS, HTML, Javascript, Python, SQL, TypeSc...",53784.0,"[c#, css, html, javascript, python, sql, types..."
6086,python,"Java, PHP, Python, SQL",82000.0,"[java, php, python, sql]"
6087,python,Python,120000.0,[python]
6089,python,"Bash/Shell, Python, SQL",130000.0,"[bash/shell, python, sql]"


# Exercise 2 - Densities and various variables

Answer the general question: **What tools (practical and theoretical) are useful to explore the base, discover patterns, associations?**

To do this, consider (same as the previous exercise):
1. Select the relevant columns to analyze.
2. Select the relevant rows to analyze. This includes removing extreme and erroneous values but can also focus the analysis on sub-populations.

## a) Joint density

What visual tools and models can you use to study the distribution and behavior of your data?

Choose three numeric variables and 2 categorical variables. Visualize the base according to several of the chosen variables. Can you describe in some way the behavior of your data? What tools would you use? describe

## b) Association

* We need to decide whether or not to remove the gross salary column. To make the survey simpler.
Is there a correlation between gross and net salary? What approach and measures would you use?

## c) Conditional density

Study the salary distribution according to the level of study.

Separate the population according to the level of education (choose two significant subpopulations) and graph both histograms of the variable `'salary_monthly_NET'` comparatively.
Do you consider that both variables are independent?
What would you analyze about it?

Compute measures of centralization and dispersion for each subpopulation

## d) Conditional Joint Density

Choose two numeric variables and one categorical.
Study the dispersion (scatterplot) of the two variables discriminating in color by the categorical variable.